<a href="https://colab.research.google.com/github/pidipidi/cs470_IAI_2024_Fall/blob/main/assignment_3/assignment_3_2_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS470 Assignment 3-2
The A∗ has been used for various AI applications including navigation. In this problem, you will adopt the result of the Problem 1.1 for the navigation problem given a simulated mobile robot. Throughout the gymasium environment, you command your mobile robot finds a global collision-free path from a current location to a goal. Then, you can observe a computed path from your A* implementations and the robot's tracking performance.

In detail, your A* algorithm returns a global path from a start to a goal randomly defined by the gym navigation environment. The provided dynamic window approach (DWA) enables a mobile agent to perform an online collision avoidance based on the global path. 

## 1. Initialize the colab environment
### Dependencies
We will first install dependencies and declare auxiliary libraries for visualization. Note that this code requires python 3.12.3.

In [ ]:
!pip install gymnasium==0.29.1 &> /dev/null
!pip install numpy==1.26.4 &> /dev/null
!pip install pygame==2.5.2 &> /dev/null
!pip install pytest==8.2.2 &> /dev/null
!pip install coverage==7.5.3 &> /dev/null
!pip install pylint==3.2.3 &> /dev/null
!pip install mypy==1.10.0 &> /dev/null
!pip install scikit-learn &> /dev/null
!pip install opencv-python &> /dev/null
!pip install pyvirtualdisplay &> /dev/null
!sudo apt-get install xvfb &> /dev/null

### Installation of the gym_navigation environment
This assignment uses a gym navigation environment where a mobile robot equipped with LiDAR sensors navigates to reach a goal while avoiding obstacles. To install the gym environment, download the [assignment3](http://github.com/pidi/CS470_IAI_2024_Fall) files from the class GitHub repository to your local machine. Run the code block below to upload the necessary files to your Colab environment.
- gym_navigation.zip
- visualization.py
- dynamic_window_approach.py

In [ ]:
from google.colab import files
uploaded = files.upload()

Now, let's install the navigation gym environment extracting it from "gym_navigation.zip"

After installing, restart_runtime() function will kill your runtime.

You can keep your run starting from the next codeblock (from 2. Implement an A* Algorithm).

In [ ]:
!pwd
# Check routine
import os, sys
if os.path.isdir("gym_navigation") is True:
    !rm -rf gym_navigation &> /dev/null
!mkdir gym_navigation &> /dev/null
!unzip gym_navigation.zip -d gym_navigation &> /dev/null
!cd gym_navigation &> /dev/null
!pip install -e gym_navigation &> /dev/null

def restart_runtime():
    os.kill(os.getpid(), 9)
restart_runtime()

## 2. Implement an A* Algorithm

### Define auxiliary class and functions

In [ ]:
import numpy as np

class Node:                                                                                                                                                                                                                                                                                 
    """                                                                                                                                                                                                                                                                                     
    Node class for the dijkstra search                                                                                                                                                                                                                                                          
    """                                                                                                                                                                                                                                                                                     
    def __init__(self, pos, idx, cost, h, prev_idx):                                                                                                                                                                                                                                           
        self.pos  = np.array(pos)                                                                                                                                                                                                                                                           
        self.idx  = idx                                                                                                                                                                                                                                                                     
        self.cost = cost   
        self.h    = h
        self.prev_idx = prev_idx # previous node's index                                                                                                                                                                                                                                    
                                                                                                                                                                                                                                                                                            
    def __str__(self):                                                                                                                                                                                                                                                                      
        return str(self.x) + "," + str(self.cost) + "," + str(self.prev_idx)   

def get_grid_index(state, resolution, grid_limits, grid_dim):                                                                                                                                                                                                                              
    """                                                                                                                                                                                                                                                                                    
    A function to convert a state to the index of grid.                                                                                                                                                                                                                                    
                                                                                                                                                                                                                                                                                           
    Parameters                                                                                                                                                                                                                                                                             
    ----------                                                                                                                                                                                                                                                                             
    state : list                                                                                                                                                                                                                                                                           
        a list of coordinate values                                                                                                                                                                                                                                                        
    resolution : float                                                                                                                                                                                                                                                                     
        a value of grid resolution                                                                                                                                                                                                                                                         
    grid_limits : list                                                                                                                                                                                                                                                                     
        a list of minimum and maximum configuration limits, where                                                                                                                                                                                                                          
        each limit is a list of float values.                                                                                                                                                                                                                                              
        (e.g., [[min_x,_min_y,min_z],[max_x,max_y,max_z]])                                                                                                                                                                                                                                 
    grid_dim : list                                                                                                                                                                                                                                                                        
        a list of grid dimensions.                                                                                                                                                                                                                                                         
                                                                                                                                                                                                                                                                                           
    Returns                                                                                                                                                                                                                                                                                
    -------                                                                                                                                                                                                                                                                                
     : Integer                                                                                                                                                                                                                                                                             
        The index of the input state in grid.                                                                                                                                                                                                                                              
    """                                                                                                                                                                                                                                                                                    
    ids = np.round((state-grid_limits[0])/resolution).astype(int)                                                                                                                                                                                                                          
    if len(state)==2:                                                                                                                                                                                                                                                                      
        return ids[0]*grid_dim[1]+ids[1]                                                                                                                                                                                                                                                   
    elif len(state)==3:                                                                                                                                                                                                                                                                    
        return ids[0]*grid_dim[1]*grid_dim[2]+ids[1]*grid_dim[2]+ids[2]                                                                                                                                                                                                                    
    return NotImplemented                                                                    

def is_valid(state, grid_limits, obstacle_tree, robot_size):                                                                                                                                                                                
    """                                                                                                                                                                                                                                                                                    
    A function to check the validity of the input state.                                                                                                                                                                                                                                   
                                                                                                                                                                                                                                                                                           
    Parameters                                                                                                                                                                                                                                                                             
    ----------                                                                                                                                                                                                                                                                             
    state : list                                                                                                                                                                                                                                                                          
        a list of coordinate values                                                                                                                                                                                                                                                       
    grid_limits : list                                                                                                                                                                                                                                                                  
        a list of minimum and maximum configuration limits, where                                                                                                                                                                                                                        
        each limit is a list of float values.                                                                                                                                                                                                                                           
        (e.g., [[min_x,_min_y,min_z],[max_x,max_y,max_z]])                                                                                                                                                                                                                               
    obstacle_tree : BallTree                                                                                                                                                                                                                                                             
        a BallTree object from Scikit-Learn. The tree includes                                                                                                                                                                                                                           
        the coordinates of obstacles.                                                                                                                                                                                                                                                    
    robot_size : float                                                                                                                                                                                                                                                                   
        a radius of a robot. This value is used for collision checking.                                                                                                                                                                                                                   
                                                                                                                                                                                                                                                                                        
    Returns                                                                                                                                                                                                                                                                                
    -------                                                                                                                                                                                                                                                                                
     : boolean                                                                                                                                                                                                                                                                             
        True if the input state is valid                                                                                                                                                                                                                                                   
    """                                                                                                                                                                                                                                                                                    
    # check workspace limits                                                                                                                                                                                                                                                               
    if any( state[i] < grid_limits[0][i] for i in range(len(grid_limits[0])) ):                                                                                                                                                                                                            
        return False                                                                                                                                                                                                                                                                       
    if any( state[i] > grid_limits[1][i] for i in range(len(grid_limits[1])) ):                                                                                                                                                                                                           
        return False                                                                                                                                                                                                                                                                      
                                                                                                                                                                                                                                                                                         
    # check collision                                                                                                                                                                                                                                                                   
    if len(np.shape(state))==1: state=state[np.newaxis,:]                                                                                                                                                                                                                                
    count = obstacle_tree.query_radius(state, robot_size, count_only=True)                                                                                                                                                                                                              
    if count>0: return False                                                                                                                                                                                                                                                       
    return True     

### Define the A* planning function (the same as problem 1)

In [ ]:
def astar_planning(start, goal, actions, resolution, grid_limits,                            
                          obstacle_tree, robot_size, **kwargs):                                 
    """                                                                                         
    A function to generate a path from a start to a goal                                        
    using A* search-based planning algorithm.                                                                                                                                                                                          
    Parameters                                                                                  
    ----------                                                                                  
    start : list                                                                                
        a list of start coordinate values (e.g., [x,y,z]).                                      
    goal : list                                                                                 
        a list of goal coordinate values (e.g., [x,y,z]).                                       
    actions : list                                                                              
        a list of available actions, where each action is a list                                
        of float values (e.g., [[vx,vy,vz],[vx,vy,vz], ..., ]).                                 
    resolution : float                                                                          
        a value of grid resolution                                                              
    grid_limits : list                                                                          
        a list of minimum and maximum configuration limits, where                               
        each limit is a list of float values.                                                   
        (e.g., [[min_x,_min_y,min_z],[max_x,max_y,max_z]])                                      
    obstacle_tree : BallTree                                                                    
        a BallTree object from Scikit-Learn. The tree includes                                  
        the coordinates of obstacles.                                                           
    robot_size : float                                                                          
        a radius of a robot. This value is used for collision checking.                                                                                                                                                                     
    Returns                                                                                     
    -------                                                                                     
    path : list                                                                                 
        a list of coordinate values that is the shortest path from                              
        the start to the goal.                                                                  
    """                                                                                         
    grid_dim = np.round((grid_limits[1]-grid_limits[0])/resolution+1).astype(int)                                                                                                                                                            
    openset, closedset = dict(), dict()                                                                                                                                                                                                      
    # Set a start node                                                                          
    start_node = Node(start,                                                                    
                      get_grid_index(start, resolution, grid_limits, grid_dim),                 
                      0, np.linalg.norm(np.array(start)-goal), -1)                                                                    
    openset[start_node.idx] = start_node                                                                                                                                                                                                     
    # Set a goal node                                                                           
    goal_node = Node(goal,                                                                      
                     get_grid_index(goal, resolution, grid_limits, grid_dim),                   
                     0, 0, -1)                                                                                                                                                                                                                  
    if start_node.idx==goal_node.idx: return []                                                 
    print("Start and goal indices: {} and {}".format(start_node.idx, 
                                                    goal_node.idx)) 
                                                    
    while True:               
        #------------------------------------------------------------
        # PLACE YOUR CODE HERE                                            
        # You need to implement according to the pseudo-code provided
        #------------------------------------------------------------
        #
        # HINT
        # ----
        # cur_idx  = GET_A_INDEX_OF_NODE_WITH_MINIMUM_HEURISTICS_FROM_OPENSET
        # cur_node = GET_A_NODE_WITH_THE_INDEX_FROM_OPENSET        
        #
 
        
        
        
        
        
        
        #------------------------------------------------------------
        
        # Remove the item from the open set                               
        del openset[cur_idx]                                              
        
        # Add it to the closed set                 
        closedset[cur_idx] = cur_node    
        
        # expand nodes based on available actions  
        for i, action in enumerate(actions):  

            #------------------------------------------------------------
            # PLACE YOUR CODE HERE
            # You need to implement according to the pseudo-code provided
            #------------------------------------------------------------
            #
            # HINT
            # ----
            # next_pos = NEXT_POSITION_BY_THE_ACTION
            # if IS_NEXT_POS_VALID:
            #   ...
            #
            # node = DEFINE_NODE_WITH_F_VALUE
            # ...
            # ADD_THE_NODE_TO_OPENSET





            
            #------------------------------------------------------------ 

    # Track the path from goal to start                                                                                                          path = [goal_node.pos]                                                                                                                      
    #------------------------------------------------------------                                                                                # ADD YOUR CODE                                                                                                                              #------------------------------------------------------------                                                                                prev_idx = goal_node.prev_idx                                                                                                                while prev_idx != start_node.idx:                                                                                                                n = closedset[prev_idx]                                                                                                                      path.append(n.pos)                                                                                                                           prev_idx = n.prev_idx                                                                                                                    #------------------------------------------------------------                                                                                return path[::-1]                           
    path = [goal_node.pos]   
                
    #------------------------------------------------------------                                                                                                                                                                                                                           
    # PLACE YOUR CODE HERE                                                                                                                                                                                                                                                                        
    #------------------------------------------------------------                                                                                                                                                                                                                           




    
    #------------------------------------------------------------                                                                                                                                                                                                                           
    return path[::-1]                            

## 3. Test your A* planning algorithm
### Run the A* planner and track the path using a DWA controller 

In [ ]:
import dynamic_window_approach as dwa
from sklearn.neighbors import BallTree
import gymnasium as gym

from moviepy.video.io.ImageSequenceClip import ImageSequenceClip                                                                                          
from pyvirtualdisplay import Display     

env = gym.make('gym_navigation:NavigationGrid-v0',                                                                                                            
               render_mode='rgb_array',                                                                                                                           
               track_id=2)                                                                                                                                    
observation, info = env.reset(seed=41)                                                                                                                        
                                                                                                                                                              
config = dwa.Config()                                                                                                                                         
# set robot [10 hz control]                                                                                                                                   
config.max_speed = 1.0 # [m/s]                                                                                                                                
config.min_speed = -0.5 # [m/s]                                                                                                                               
config.max_yaw_rate = 40.0/180.0*np.pi  # [rad/s]                                                                                                             
config.max_accel = 0.1 # [m/ss]                                                                                                                               
config.max_delta_yaw_rate = 40.0/180.0*np.pi  # [rad/ss]                                                                                                      
config.v_resolution = 0.1  # [m/s] #used?                                                                                                                     
config.yaw_rate_resolution = 0.1/180.0*np.pi # [rad/s] #used?                                                                                                 
config.dt = 0.1  # [s] Time tick for motion prediction                                                                                                        
config.predict_time = 10.0  # [s]                                                                                                                             
config.to_goal_cost_gain = 0.15                                                                                                                               
config.speed_cost_gain = 1.0                                                                                                                                  
config.obstacle_cost_gain = 1.0                                                                                                                               
config.robot_stuck_flag_cons = 0.001  # constant to prevent robot stucked                                                                                     
                                                                                                                                                              
config.robot_type   = dwa.RobotType.circle                                                                                                                    
config.robot_radius = 0.4                                                                                                                                     
                                                                                                                                                              
action = None                                                                                                                                                 
#actions   = [[-0.5,0], [0,-0.5], [0.5,0], [0,0.5]]                                                                                                           
actions = [[-0.5,0], [0,-0.5], [0.5,0], [0,0.5],                                                                                                              
           [-0.5,-0.5],[-0.5,0.5],[0.5,-0.5],[0.5,0.5],]                                                                                                      
grid_size = 0.5                                                                                                                                               
ob = None                                                                                                                                                     
terminated = True   

display = Display(visible=0, size=(300, 300))                                                                                                             
display.start() 

env_name = env.unwrapped.spec.id                                                                                                                          
suffix   = '_astar'
filename = env_name + suffix + ".mp4"                                                                                                                     
recorded_frames = []   

for i in range(4000):   
    # Record your agent behavior
    frame = env.unwrapped._render_frame("rgb_array")    
    if i%8==0:
        recorded_frames.append(frame) 
    
    config.ob = np.array(env.get_wrapper_attr('obstacles_centers'))                                                                                                                                                                          
                                                                                                                                                                                                                                             
    # Set a goal                                                                                                                                                                                                                             
    goal = env.get_wrapper_attr('_goal')                                                                                                                                                                                                     
    goal = np.array([goal.x_coordinate, goal.y_coordinate])                                                                                                                                                                                  
                                                                                                                                                                                                                                             
    # Get the current position                                                                                                                                                                                                               
    # initial state [x(m), y(m), yaw(rad), v(m/s), omega(rad/s)]                                                                                                                                                                             
    # where gym_nav                                                                                                                                                                                                                          
    # 1) y-direction is 0 angle                                                                                                                                                                                                              
    # 2) positive as clockwise direction.                                                                                                                                                                                                    
    pose = env.get_wrapper_attr('_pose')                                                                                                                                                                                                     
    if terminated:                                                                                                                                                                                                                           
        x = np.array([pose.position.x_coordinate,                                                                                                                                                                                            
                      pose.position.y_coordinate,                                                                                                                                                                                            
                      -(pose.yaw-np.pi/2.0),                                                                                                                                                                                                 
                      0,                                                                                                                                                                                                                     
                      0])                                                                                                                                                                                                                    
    else:                                                                                                                                                                                                                                    
        x = np.array([pose.position.x_coordinate,                                                                                                                                                                                            
                      pose.position.y_coordinate,                                                                                                                                                                                            
                      -(pose.yaw-np.pi/2.0),                                                                                                                                                                                                 
                      action[0]/2,                                                                                                                                                                                                           
                      -action[1]/2                                                                                                                                                                                                           
                      ])                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
                                                                                                                                                                                                                                      
    if terminated:                                                                                                                                                                                                                          
        start = x[:2]                                                                                                                                                                                                                        
        start[0] = round(start[0]*2)/2                                                                                                                                                                                                       
        start[1] = round(start[1]*2)/2                                                                                                                                                                                                       
                                                                                                                                                                                                                                             
        grid_limits = np.array([[env.get_wrapper_attr('_track').spawn_area[0][0][0],                                                                                                                                                         
                                 env.get_wrapper_attr('_track').spawn_area[0][1][0]],                                                                                                                                                        
                                [env.get_wrapper_attr('_track').spawn_area[0][0][1],                                                                                                                                                         
                                 env.get_wrapper_attr('_track').spawn_area[0][1][1]]]                                                                                                                                                        
                               )                                                                                                                                                                                                             
                                                                                                                                                                                                                                             
        obstacle_tree = BallTree(config.ob)                                                                                                                                                                                                  
        robot_size = config.robot_radius+0.75                                                                                                                                                                                                

        # Find a global path using your A* algorithm
        global_path = astar_planning(start, goal, actions,                                                                                                                                                                       
                                                 grid_size, grid_limits,                                                                                                                                                                     
                                                 obstacle_tree, robot_size)      
    # Find the nearest point on the global path        
    cut_flag = False                                                                                                                                                                                                                         
    for j in range(len(global_path)):                                                                                                                                                                                                        
        if np.linalg.norm(x[:2]-global_path[j]) <= config.robot_radius:                                                                                                                                                                      
            for k in range(j+1):                                                                                                                                                                                                             
                global_path.pop(0)                                                                                                                                                                                                           
            break                                                                                                                                                                                                                            

    # Run a DWA controller to track the path
    temp_goal = global_path[0]                                                                                                                                                                                                               
    action, predicted_trajectory = dwa.dwa_control(x, config, temp_goal, config.ob)                                                                                                                                                          
    action[1] *= -1.0                                                                                                                                                                                                                        
                                                                                                                                                                                                                                             
    env.unwrapped.set_predicted_traj(predicted_trajectory)                                                                                                                                                                                             
    env.unwrapped.set_global_traj(global_path)                                                                                                                                                                                                         

    # Apply the action to the environment
    observation, reward, terminated, truncated, info = env.unwrapped.step_raw_action(action)                                                                                                                                                           
                                                                                                                                                                                                                                             
    if terminated:                                                                                                                                                                                                                           
        observation, info = env.reset()                                                                                                                                                                                                      
        print(f'observation={observation} info={info}')                                                                                                                                                                                      
                                                                                                                                                                                                                                             
env.close()  

clip = ImageSequenceClip(recorded_frames, fps=8)                                                                                                          
clip.write_videofile(filename, logger=None)  

### Playback the recorded navigation video!

In [ ]:
from visualization import *
from IPython.display import HTML

HTML(play_video(filename))